In [1]:
import argparse
from datetime import datetime
import json
import logging
import os
import math
import sys
import time
import topologyqueries,linkedlist,update_topology,spanning_tree
import topo_meas_queries, process_network, linknet, update_topology

from gridappsd import GridAPPSD, DifferenceBuilder, utils, topics as t
from gridappsd.topics import simulation_input_topic, simulation_log_topic, simulation_output_topic


#model_mrid = sim_request["power_system_config"]["Line_name"]
#model_mrid = "_C125761E-9C21-4CA9-9271-B168150DE276" #ieee13training
#model_mrid = "_EE71F6C9-56F0-4167-A14E-7F4C71F10EAA" #final9500node
#model_mrid = "_AAE94E4A-2465-6F5E-37B1-3E72183A4E44" #test9500
model_mrid = "_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"  # 13 bus
#model_mrid = "_5B816B93-7A5F-B64C-8460-47C17D6E4B0F" #ieee13assets
#model_mrid="_C1C3E687-6FFD-C753-582B-632A27E28507" # IEEE 123
#model_mrid="_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62"

requestedTime=1570041125



global BaseConnDict,BaseTermDict,TermList, NodeList
global XfmrKeys,XfmrDict,SwitchKeys,SwitchDict,DG_query

os.environ['GRIDAPPSD_USER'] = 'app_user'
os.environ['GRIDAPPSD_PASSWORD'] = '1234App'

gapps = GridAPPSD()
assert gapps.connected

In [2]:
[ConnNodeDict, EquipDict] = process_network.build_equip_dicts(gapps, model_mrid)


Processed  22 ConnectivyNode objects in  0.0024041800000000335 seconds
Processed  183 Measurement objects in  0.005794752000000014 seconds
Processed  11 ACLineSegment objects in  0.001635031000000009 seconds
Processed  14 Transformer objects in  0.0016641479999999764 seconds
Processed  5 Switch objects in  0.0014263529999999913 seconds
Processed  0 House objects in  0.001209479999999985 seconds
Processed  3 RatioTapChanger objects in  0.0012802339999999912 seconds


In [3]:
BaseConnDict = json.dumps(ConnNodeDict)
MVConnNodeDict = json.loads(BaseConnDict)

In [4]:
#[MVConnNodeDict, EquipDict] = process_network.build_equip_dicts(gapps, model_mrid)


In [5]:
EqTypes = ['ACLineSegment', 'PowerTransformer', 'TransformerTank', 'SynchronousMachine']
[ConnNodeDict, TerminalsDict, NodeList, TermList] = linknet.build_linknet_lists(ConnNodeDict, EquipDict, EqTypes)

Processed  11 ACLineSegment objects in  4.6608000000003535e-05 seconds
Processed  2 PowerTransformer objects in  2.2931999999975528e-05 seconds
Processed  4 TransformerTank objects in  2.562999999999871e-05 seconds
Processed  0 SynchronousMachine objects in  1.0039000000017229e-05 seconds
Processed  1 missing nodes in  2.104900000005072e-05 seconds


In [6]:
EqTypes = ['ACLineSegment', 'RatioTapChanger', 'SynchronousMachine']
[MVConnNodeDict, MVTerminalsDict, MVNodeList, MVTermList] = linknet.build_linknet_lists(MVConnNodeDict, EquipDict, EqTypes)

Processed  11 ACLineSegment objects in  3.805600000000853e-05 seconds
Processed  1 RatioTapChanger objects in  1.677399999999718e-05 seconds
Processed  0 SynchronousMachine objects in  1.1774999999991653e-05 seconds
Processed  6 missing nodes in  2.3657000000010253e-05 seconds


In [7]:
SwitchDict = {}
SwitchDict.update(EquipDict['Breaker'])
SwitchDict.update(EquipDict['Fuse'])
SwitchDict.update(EquipDict['LoadBreakSwitch'])
SwitchDict.update(EquipDict['Recloser'])
SwitchKeys = list(SwitchDict.keys())

In [8]:
MVTrees = {}
[MVTrees, MVConnNodeDict] = spanning_tree.local_spanning_tree(MVConnNodeDict, MVTerminalsDict, MVNodeList, MVTermList, EquipDict, 'Breaker', list(EquipDict['Breaker'].keys()), MVTrees, 'all')
[MVTrees, MVConnNodeDict] = spanning_tree.local_spanning_tree(MVConnNodeDict, MVTerminalsDict, MVNodeList, MVTermList, EquipDict, 'Fuse', list(EquipDict['Fuse'].keys()), MVTrees, 'single')
[MVTrees, MVConnNodeDict] = spanning_tree.local_spanning_tree(MVConnNodeDict, MVTerminalsDict, MVNodeList, MVTermList, EquipDict, 'LoadBreakSwitch', list(EquipDict['LoadBreakSwitch'].keys()), MVTrees, 'all')
[MVTrees, MVConnNodeDict] = spanning_tree.local_spanning_tree(MVConnNodeDict, MVTerminalsDict, MVNodeList, MVTermList, EquipDict, 'Recloser', list(EquipDict['Recloser'].keys()), MVTrees, 'single')


Processed topology from   _52DE9189-20DC-4C73-BDEE-E960FE1F9493  with  1  buses
Processed  1 topology trees containing  1  buses in  0.0006250709999999771 seconds
Processed topology from   _43EF8365-F932-409B-A51E-FBED3F6DFFAA  with  6  buses
Processed  1 topology trees containing  6  buses in  0.00016876799999998582 seconds
Processed topology from   _2858B6C2-0886-4269-884C-06FA8B887319  with  6  buses
Processed topology from   _517413CB-6977-46FA-8911-C82332E42884  with  2  buses
Processed  2 topology trees containing  8  buses in  0.0030798149999999858 seconds
Processed topology from   _CE5D0651-676B-4AF3-8D67-41BF1B33E30C  with  6  buses
Processed  1 topology trees containing  6  buses in  0.00020410399999998274 seconds


In [61]:
DistAppStruct = {}
DistAppStruct['feeders'] = {}
DistAppStruct['feeders'][model_mrid] = {}
DistAppStruct['feeders'][model_mrid]['addressable_equipment'] = []
DistAppStruct['feeders'][model_mrid]['unaddressable_equipment'] = []
DistAppStruct['feeders'][model_mrid]['switch_areas'] = []
ProcessedNodes = []
XfmrDict = EquipDict['PowerTransformer']
XfmrDict.update(EquipDict['TransformerTank'])

In [62]:
for i1 in range(len(SwitchKeys)):
    SwitchArea = {}
    SwitchArea['boundary_switches'] = []
    SwitchArea['addressable_equipment'] = []
    SwitchArea['unaddressable_equipment'] = []
    SwitchArea['connectivity_node'] = []
    SwitchArea['secondary_areas'] = []
    #print(EquipDict['LoadBreakSwitch'][SwitchKeys[i1]]['name'])
    for i2 in range(len(MVTrees[SwitchKeys[i1]])):
        node = MVTrees[SwitchKeys[i1]][i2]
        print('mv node', ConnNodeDict[node]['name'], node)
        ProcessedNodes.append(node)
        SwitchArea['boundary_switches'].extend(ConnNodeDict[node]['Breaker'])
        SwitchArea['boundary_switches'].extend(ConnNodeDict[node]['Fuse'])
        SwitchArea['boundary_switches'].extend(ConnNodeDict[node]['LoadBreakSwitch'])
        SwitchArea['boundary_switches'].extend(ConnNodeDict[node]['Recloser'])
        #SwitchArea['addressable_equipment'].extend(ConnNodeDict[node]['SynchronousMachine'])
        #SwitchArea['addressable_equipment'].extend(ConnNodeDict[node]['PowerElectronicsConnection'])
        #SwitchArea['addressable_equipment'].extend(ConnNodeDict[node]['LinearShuntCompensator'])
        #SwitchArea['addressable_equipment'].extend(ConnNodeDict[node]['RatioTapChanger'])
        #SwitchArea['unaddressable_equipment'].extend(ConnNodeDict[node]['ACLineSegment'])
        #SwitchArea['unaddressable_equipment'].extend(ConnNodeDict[node]['PowerTransformer'])
        #SwitchArea['unaddressable_equipment'].extend(ConnNodeDict[node]['TransformerTank'])
        #SwitchArea['unaddressable_equipment'].extend(ConnNodeDict[node]['Measurement'])
        SwitchArea['connectivity_node'].append(ConnNodeDict[node]['name'])
        DistXfmrs = ConnNodeDict[node]['TransformerTank'] #+ ConnNodeDict[node]['PowerTransformer']
        if DistXfmrs:
            SwitchArea['unaddressable_equipment'].extend(DistXfmrs)
            for i3 in range(len(DistXfmrs)):
                xfmr = DistXfmrs[i3]
                [LVTree, LVConnNodeDict] = spanning_tree.local_spanning_tree(ConnNodeDict, TerminalsDict, NodeList, TermList, EquipDict, 'TransformerTank', [xfmr],  MVTrees, 'all')
                LVTreeKeys = LVTree[xfmr]
                if LVTreeKeys: 
                    LVTreeKeys.pop(0)
                    DistArea = {}
                    DistArea['distribution_transformer'] = [xfmr]
                    DistArea['addressable_equipment'] = []
                    DistArea['unaddressable_equipment'] = []
                    DistArea['connectivity_node'] = []
                    for i4 in range(len(LVTreeKeys)):
                        lvnode = LVTreeKeys[i4]
                        ProcessedNodes.append(lvnode)
                        print('lvnode ', ConnNodeDict[lvnode]['name'], lvnode)
                        #DistArea['addressable_equipment'].extend(ConnNodeDict[lvnode]['SynchronousMachine'])
                       # DistArea['addressable_equipment'].extend(ConnNodeDict[lvnode]['PowerElectronicsConnection'])
                        #DistArea['addressable_equipment'].extend(ConnNodeDict[lvnode]['EnergyConsumer'])
                       # DistArea['addressable_equipment'].extend(ConnNodeDict[lvnode]['House'])
                        
                        #DistArea['unaddressable_equipment'].extend(ConnNodeDict[lvnode]['ACLineSegment'])
                        #DistArea['unaddressable_equipment'].extend(ConnNodeDict[lvnode]['Measurement'])
                       # DistArea['connectivity_node'].append(lvnode)
                        DistArea['connectivity_node'].append(ConnNodeDict[lvnode]['name'])
                        #SwitchArea['unaddressable_equipment'].extend(ConnNodeDict[lvnode]['ACLineSegment'])
                        #SwitchArea['unaddressable_equipment'].extend(ConnNodeDict[lvnode]['Measurement'])
                    
            SwitchArea['secondary_areas'].append((DistArea.copy()))
            DistArea = {}
    DistAppStruct['feeders'][model_mrid]['switch_areas'].append(dict(SwitchArea))
    SwitchArea = {}
        #DistAppStruct['feeders'][model_mrid]['unaddressable_equipment'].extend(SwitchArea['unaddressable_equipment'])
        
AllNodes = list(ConnNodeDict.keys())
MissingNodes = list(set(AllNodes).difference(ProcessedNodes))
for i5 in range(len(MissingNodes)):
    node = MissingNodes[i5]
    DistAppStruct['feeders'][model_mrid]['addressable_equipment'].extend(ConnNodeDict[node]['SynchronousMachine'])
    DistAppStruct['feeders'][model_mrid]['addressable_equipment'].extend(ConnNodeDict[node]['PowerElectronicsConnection'])
    DistAppStruct['feeders'][model_mrid]['addressable_equipment'].extend(ConnNodeDict[node]['LinearShuntCompensator'])
    DistAppStruct['feeders'][model_mrid]['addressable_equipment'].extend(ConnNodeDict[node]['RatioTapChanger'])
    DistAppStruct['feeders'][model_mrid]['unaddressable_equipment'].extend(ConnNodeDict[node]['ACLineSegment'])
    DistAppStruct['feeders'][model_mrid]['unaddressable_equipment'].extend(ConnNodeDict[node]['PowerTransformer'])
    DistAppStruct['feeders'][model_mrid]['unaddressable_equipment'].extend(ConnNodeDict[node]['TransformerTank'])
    DistAppStruct['feeders'][model_mrid]['unaddressable_equipment'].extend(ConnNodeDict[node]['Measurement'])

mv node 650 _7BEDDADD-0A14-429F-8601-9EA8B892CA6E
mv node 633 _C6256170-E6ED-4F91-8EBD-748090C3FDD5
mv node 632 _6CB5E5CE-2CD0-40CC-A979-B4F9ED05E49B
mv node rg60 _673E896A-DCBF-4E43-9924-BEB31C5B6005
Processed  0 topology trees containing  0  buses in  1.0372999999841426e-05 seconds
Processed  0 topology trees containing  0  buses in  1.2976999999914085e-05 seconds
Processed  0 topology trees containing  0  buses in  1.1797999999618725e-05 seconds
mv node 645 _0124E881-B82D-4206-BBDF-37D585159872
mv node brkr _94F822E0-7130-4205-8597-B47110BBEF4B
Processed  0 topology trees containing  0  buses in  5.15199999995275e-06 seconds
Processed  0 topology trees containing  0  buses in  9.425000000007344e-06 seconds
Processed  0 topology trees containing  0  buses in  9.155999999954645e-06 seconds
mv node 646 _8C58660F-C62C-4903-BE72-22F1255B1E62
mv node 684 _8E99F99D-FE8F-420B-AC49-0B52DF5362AB
mv node 671 _30BE5988-DE57-4E0C-AB08-50D5A13D2C1B
mv node 652 _0F1E28C3-6C44-4F88-B79C-2FDBCA4487B

In [38]:
from gridappsd import topics as t
topic = t.REQUEST_POWERGRID_DATA

object_mrid = "_2858B6C2-0886-4269-884C-06FA8B887319"

message = {
    "requestType": "QUERY_OBJECT_DICT",
    "resultFormat": "JSON",
    "modelId": '_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62',
    "objectId": '_e92f7019-68ec-4ee5-8fc6-e66580f24794'
}

gapps.get_response(topic, message)

{'data': [], 'responseComplete': True, 'id': '1606464209'}

In [12]:
for i0 in range(len(SwitchKeys)):
    print(SwitchDict[SwitchKeys[i0]]['name'])
    for i8 in range(len(MVTrees[SwitchKeys[i0]])):
        
        print(ConnNodeDict[MVTrees[SwitchKeys[i0]][i8]]['name'])

brkr1
650


KeyError: 'name'

In [21]:
import sys
sys.getsizeof(DistAppStruct)

232

In [63]:
DistAppStruct

{'feeders': {'_49AD8E07-3BF9-A4E2-CB8F-C3722F837B62': {'addressable_equipment': ['_9D725810-BFD6-44C6-961A-2BC027F6FC95',
    '_D2E930A7-B136-4ACA-A996-8DB5C60AADF3',
    '_7B671984-4C56-4FF1-9733-B4B6FCA5F2AA'],
   'unaddressable_equipment': ['_259E820F-B4AF-4E1A-8271-687534EDAECC',
    '_c5545620-2c8c-42ff-b59d-33cd7e02b8e5',
    '_dda6be90-3cd1-4a27-a691-3f95aa4e4e13',
    '_f3cc955e-8013-447a-848e-4c7af516cb34',
    '_7bbbda89-9830-472c-82bc-4e4f599369bb',
    '_a870a258-34be-46b8-a4de-faee32275a9a',
    '_bc0d2bad-2c6b-4266-8c47-b32a536288f0',
    '_8E180773-2A9B-4136-BC9A-132A52C8E276',
    '_fad827c5-f344-498e-ab39-29883d4c6bb6',
    '_7a84c113-d427-4802-a769-b6edc48451c7',
    '_162b4555-f748-4829-8e71-bbbecf4e4bcb',
    '_eb606875-f620-4c4e-b559-cd587ba0ecf8',
    '_6c08f0e8-eb6f-4261-a3f6-3aa9d1e99c75',
    '_1E6B5C97-C4E8-4CED-B9A5-6E69F389DA93',
    '_5ae00771-adeb-4ee2-92a3-7c991c579ce1',
    '_77cf5e05-b332-4203-937a-53f58b4f4585',
    '_247f50ee-6bc5-4588-bdf1-9b480e0984

In [26]:

with open('output_message.json', 'w') as f:
    json.dump(DistAppStruct, f)